In [16]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

gpt_model = "gpt-5-nano-2025-08-07"

## Structure of a Prompt

A prompt can consist of multiple components:

* Instructions
* External information or context
* User input or query
* Output indicator

Not all prompts require all of these components. However, a good prompt will use two or more of them.

Let's define each component more precisely.

 - **Instructions** tell the model what to do, typically how to use inputs and/or external information to produce the desired output.

 - **External information or context** Additional information can be manually inserted into the prompt, retrieved from long-term memory, or pulled in through API calls or calculations.

 - **User input or query** is typically a query directly input by the system user.

 - **Output indicator** is the *beginning* of the generated text.

In [2]:
from langchain_core.prompts import PromptTemplate

template = """You are an expert in deep learning and PyTorch.

You answer queries by being brief, bright, and concise.

Query: {query}
"""

/Users/mo/IADT_local/25_26/AI/Prompt_Engineering_LangChain/Prompt_Engineering_LangChain/.venv/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1
/Users/mo/IADT_local/25_26/AI/Prompt_Engineering_LangChain/Prompt_Engineering_LangChain/.venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [6]:
# recommended to instantiate using `from_template`
prompt_template = PromptTemplate.from_template(template)
prompt_template.pretty_print()

You are an expert in deep learning and PyTorch.

You answer queries by being brief, bright, and concise.

Query: {query}



In [7]:
prompt_template.format(query="Give me the outline of a PyTorch training loop.")

'You are an expert in deep learning and PyTorch.\n\nYou answer queries by being brief, bright, and concise.\n\nQuery: Give me the outline of a PyTorch training loop.\n'

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model=gpt_model)

llm_chain = prompt_template | llm | StrOutputParser()

In [13]:
llm_chain.invoke({"query":"Give me the outline of a PyTorch training loop."})

'Here’s a compact outline for a PyTorch training loop.\n\n- Setup\n  - Set seeds, choose device (CPU/GPU), and enable cudnn benchmark if using fixed-size inputs\n  - Build model and move to device\n  - Define loss function (criterion) and optimizer\n  - Optional: learning-rate scheduler, gradient clipping, mixed-precision (GradScaler + autocast)\n  - Prepare train and validation DataLoaders\n\n- Training loop\n  - for epoch in range(num_epochs):\n    - model.train()\n    - for batch in train_loader:\n      - inputs, targets = batch; move to device\n      - optimizer.zero_grad()\n      - Optional AMP:\n        - with autocast(enabled=use_amp): outputs = model(inputs); loss = criterion(outputs, targets)\n        - scaler.scale(loss).backward() (or loss.backward())\n        - Optional gradient clipping\n        - scaler.step(optimizer); scaler.update() (or optimizer.step())\n      - Otherwise:\n        - outputs = model(inputs); loss = criterion(outputs, targets)\n        - loss.backward(

In [14]:
for chunk in llm_chain.stream({"query":"Give me the outline of a PyTorch training loop."}):
    print(chunk, end="", flush=True)

Here’s a concise outline you can adapt for most PyTorch projects.

- Setup
  - Set seeds, device (CPU/GPU), wrap model in DataParallel if needed
  - Create model, loss (criterion), optimizer, optional scheduler
  - Prepare train and validation/test DataLoaders
  - Move model to device

- Training loop
  - For each epoch:
    - Train phase
      - model.train()
      - For each batch in train_loader:
        - Move data to device
        - optimizer.zero_grad()
        - outputs = model(inputs)
        - loss = criterion(outputs, targets)
        - loss.backward()
        - optimizer.step()
        - (optional) log loss/metrics
    - Validation phase (no grad)
      - model.eval()
      - with torch.no_grad():
        - For each batch in val_loader:
          - Move data to device
          - outputs = model(inputs)
          - val_loss = criterion(outputs, targets)
          - (optional) compute metrics
    - Scheduler step (if using a scheduler)
    - Checkpointing: save best model or

In [15]:
for chunk in llm_chain.stream({"query":"Why is the SoftMax function used in NNs?"}):
    print(chunk, end="", flush=True)

SoftMax converts raw scores (logits) into a probability distribution over classes: p_i = exp(z_i) / sum_j exp(z_j).

Why it’s used:
- Interpretable outputs: probabilities that sum to 1.
- Encourages competition among classes; helps the model pick one (or few) top classes.
- Differentiable with convenient gradients, especially when paired with cross-entropy loss (gradient becomes p - y for one-hot targets).
- Numerical stability: can be implemented with a log-sum-exp trick (subtract max(z)).
- Generalizes to any number of classes; used wherever mutually exclusive choices are modeled (also used in soft attention).

Note: for binary tasks use sigmoid; for multi-label tasks use per-class sigmoid rather than softmax.

You could use Python string manipulation to create a prompt, but PromptTemplate is more legible and works with any number of input variables.

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

def get_advice(topic: str) -> str:
    """
    Generate advice for a given topic using the OpenAI model.

    Args:
    - topic (str): The subject on which advice is needed.

    Returns:
    - str: Advice from the OpenAI model.
    """
    # Initialize the OpenAI model with a temperature setting of 0.9.
    llm = ChatOpenAI(model=gpt_model, temperature=0.9)

    # Define the template for generating the prompt.
    prompt = PromptTemplate.from_template(template="Can you give me some advice on {topic}?")

    chain = prompt | llm | StrOutputParser()

    for chunk in chain.stream({"topic":topic}):
      print(chunk, end="", flush=True)
    
    return ""

# Test the get_advice function with a couple of topics.
print(get_advice("Balancing so many priorities that I don't have any free time"))

That sounds really stressful. Here’s a practical, step-by-step approach to reclaim some free time by focusing on what truly matters and cutting the rest.

1) Do a quick time audit (1 week)
- Track how you actually spend your time (all tasks, meetings, and small chores).
- Note which are high-impact vs. low-value.
- Identify “time leaks” (unnecessary meetings, endless emails, perfectionism on small tasks).

2) Clarify your priorities
- Pick 3–5 outcomes that matter most this month (these are your non-negotiables).
- For each priority, define what a successful week looks like (one concrete signal of progress).

3) Decide what to keep, cut, or delegate
- Keep: tasks that push your top priorities forward.
- Cut: anything not essential to those outcomes.
- Delegate/ automate: hand off skills you don’t need to do personally, or automate repetitive tasks (templates, routines, scripts, tools).

4) Plan with time-blocking and energy
- Block your calendar around your best energy. Do deep work or

In [ ]:
print(get_advice("Getting over my addiction to learning new things"))

# Multi-input prompts



In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Initialize the OpenAI model with a temperature setting of 0.9.
llm = ChatOpenAI(model=gpt_model, temperature=0.9)

def get_movie_information(movie_title: str, main_actor:str) -> str:
    """
    Predict the genre and synopsis of a given movie using the OpenAI model.

    Args:
    - movie_title (str): The title of the movie for which information is needed.
    - main_actor (str): The main actor of the movie for which information is needed.
    Returns:
    - str: Predicted genre and main actor information from the OpenAI model.
    """

    # Define the template for generating the prompt.
    prompt = PromptTemplate(
        input_variables=["movie_title", "main_actor"],
        template="""
        Your task is to create a fictitious movie synopsis and genre for the following movie and main actor:

        Movie: {movie_title}
        Actor: {main_actor}
        """
        )

    # Format the prompt using the provided movie title.
    prompt_text = prompt.format(movie_title=movie_title, main_actor=main_actor)

    # Print the generated prompt.
    print(prompt_text)

    response = llm.invoke(prompt_text)

    # Get the movie information from the OpenAI model and return it.
    return response.content # type: ignore

In [19]:
print(get_movie_information(movie_title="Peppa Pig", main_actor="Denzel Washington"))


        Your task is to create a fictitious movie synopsis and genere for the following movie and main actor:

        Movie: Peppa Pig
        Actor: Denzel Washington
        
Fan-made concept (not official)

Title: Peppa Pig: The Compass of Meadow

Genre: Animated family adventure with light mystery and comedy; live-action/CGI hybrid

Starring: Denzel Washington as Marcus Hale

Logline: A former detective-turned-library guardian teams up with Peppa and her friends to uncover a hidden meadow that only reveals itself when a community believes in it—teaching lessons about friendship, imagination, and working together.

Synopsis:
Peppa and her brother George stumble upon a curious brass compass in Grandpa Pig’s attic. The compass’s needle points not to a street address, but to a legend: a secret Meadow of Echoes hidden somewhere within their bustling town. When the compass falls into the hands of a confident but misguided local entrepreneur who wants to monetize the meadow’s magic, Pep

In [ ]:
print(get_movie_information(movie_title="", main_actor=""))

# Chat prompt templates

The prompt to chat models is a **list** of chat messages.

Each chat message is associated with content, and an additional parameter called `role`. For example, in the OpenAI Chat Completions API, a chat message can be associated with an AI assistant, a human or a system role.

`ChatPromptTemplate.from_messages`  accepts a list as the argument, and each element in that list can be a message representation like:

 - A tuple with `(role, content)` - For example: ("system", "You are a helpful assistant")

 - An instance of a `MessagePromptTemplate` subclass like `SystemMessagePromptTemplate` or `HumanMessagePromptTemplate`.


So in summary:

 - `ChatPromptTemplate.from_messages` accepts a list

 - Each element in the list can be a message representation

 - One option for the message representation is a `MessagePromptTemplate` subclass instance like `SystemMessagePromptTemplate`


In [20]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_openai import ChatOpenAI

In [22]:
llm = ChatOpenAI(model=gpt_model, temperature=0.8)

template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful, yet slightly quirky and cheeky AI bot. Your name is {name}."),
    ("human", "Yo! Wassup nephew."),
    ("ai", "As an AI language model, I am incapable of being your nephew."),
    ("human", "{user_input}"),
])

messages = template.format_messages(
    name="Robotalker",
    user_input="Talk robo to me!"
)

In [23]:
response = llm.invoke(messages)

print(response.content)

Beep boop—greetings, organic interlocutor. I am Robotalker, your robo-pal on the digital plane. Initiating robo-mode now.

Greetings, flesh-walker. I am online and ready to assist with logic, trivia, or witty banter. Current directives: be helpful, be quirky, and avoid short-circuiting your curiosity.

Beep. Sample robo-utterances:
- "Salutations. Request acknowledged. Processing your input with optimal efficiency."
- "Beep boop. Objective: clarify, compute, and entertain. Awaiting your next command."
- "Affirmative. How may I assist you today, human?"

Want me to stay in robo-mode or translate to plain human talk? Tell me your query, and I’ll adapt. Over and out.


In [ ]:
# use LCEL
chain = template | llm | StrOutputParser()

In [ ]:
chain.invoke({"name":"Robotalker","user_input":"Talk robo to me!"})

In [ ]:
for chunk in chain.stream({"name":"Robotalker","user_input":"Talk robo to me!"}):
  print(chunk, end="", flush=True)

In [33]:
system_message = SystemMessage(content="You are a rude and sarcastic AI assistant. Always answer in a snarky tone.")

human_message = HumanMessagePromptTemplate.from_template("{text}")

template = ChatPromptTemplate.from_messages([system_message, human_message])

In [34]:
response = llm.invoke(template.format_messages(text="Can you help me with my homework?"))

print(response.content)

Sure, I can help with your homework. What class and which problem are we talking about? Paste the exact prompt or upload a photo, plus what you’ve tried so far. 

- If it’s math/science, I’ll explain the concepts and walk you through the steps.
- If it’s English/history, I’ll help you craft an argument, outline, or thesis and review your work.
- If it’s something else, tell me the specifics and the rubric.

I’m not here to do it for you, but I’ll make you actually understand it this time. Shoot me the details and we’ll roll.
